# Flatiron Phase 4 Project

* <b>Name:</b> James Benedito
* <b>Pace:</b> Part-Time
* <b>Instructor:</b> Morgan Jones

# Business Problem

When it comes to business in general, it is important to make informed decisions on where to invest funds because a high ROI is imperative for a company to thrive. Data is a powerful tool that can be leveraged to determine where huge profits can potentially be gained.

This Jupyter notebook will examine data to highlight the Top 5 zipcodes to invest in. The information presented will be applicable to real estate companies in Nevada state who are seeking out the best locations for their clients.

# Goal

My goal is to use time series data from Zillow to generate recommendations for a theoretical real estate company in Nevada, in terms of which zipcodes would be best to invest in. My suggestions will be informed by an optimized ARIMA model. I will use ROI as my main metric of interest.

# Dataset 

The data used for this project comes from Zillow and is stored in a file called <b>zillow_data.csv</b>. The dataset shows housing prices over time, with each row representing a specific house in a particular US city. As mentioned previously, I will be focusing on merely a subset of this data. The dataframe will be filtered so that it only includes houses located in Nevada. 

# Data Preprocessing

In [1]:
# import necessary packages
import warnings
import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
from collections import Counter
import folium
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pylab import rcParams
%matplotlib inline

In [2]:
# read dataset into pandas df

zillow_df = pd.read_csv('zillow_data.csv')
zillow_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [17]:
# filter dataset so it only includes houses in Nevada

ca_df = zillow_df[zillow_df['State']=='CA']
ca_df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
9,97564,94109,San Francisco,CA,San Francisco,San Francisco,10,766000.0,771100.0,776500.0,...,3767700,3763900,3775000,3799700,3793900,3778700,3770800,3763100,3779800,3813500
15,96107,90250,Hawthorne,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,16,152500.0,152600.0,152600.0,...,579300,585700,590900,594700,598500,601300,602800,606100,612400,616200
27,97771,94565,Pittsburg,CA,San Francisco,Contra Costa,28,139200.0,138300.0,137500.0,...,394900,398400,401600,405400,408600,410900,413700,417900,424300,430100
30,96027,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,31,340600.0,341700.0,343000.0,...,1839800,1861100,1888600,1903900,1907500,1922100,1952400,1974500,1975900,1966900
64,97711,94501,Alameda,CA,San Francisco,Alameda,65,222400.0,222300.0,222400.0,...,965100,975000,987800,1000000,1009900,1021000,1032400,1042000,1053000,1059700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14683,97304,93517,Bridgeport,CA,NaN,Mono,14684,120900.0,121400.0,121800.0,...,287700,276300,263900,259000,259800,262600,264400,265900,269400,272500
14690,98404,95728,Truckee,CA,Truckee,Nevada,14691,147700.0,147800.0,148100.0,...,476900,488600,494100,491200,485700,482600,480900,483600,490500,496300
14692,98245,95497,Annapolis,CA,Santa Rosa,Sonoma,14693,307200.0,306300.0,305500.0,...,772800,778200,786100,792700,799100,806900,814600,824300,837100,848700
14709,96805,92322,Crestline,CA,Riverside,San Bernardino,14710,78700.0,78500.0,78300.0,...,180900,182600,184200,185700,187300,189100,189900,191600,195700,200100


In [21]:
ca_df['City'].value_counts().keys()

Index(['Los Angeles', 'San Diego', 'San Jose', 'Sacramento', 'San Francisco',
       'Fresno', 'Long Beach', 'Oakland', 'Bakersfield', 'Glendale',
       ...
       'San Juan Capistrano', 'Newark', 'Tulelake', 'Santa Fe Springs',
       'Agoura Hills', 'Coronado', 'Laguna Beach', 'Sebastopol', 'Anderson',
       'Hughson'],
      dtype='object', length=677)

# Step 2: Data Preprocessing

In [11]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[:], format='%Y-%m')

In [12]:
#get_datetimes(zillow_df)

ValueError: time data RegionID doesn't match format specified

# Step 3: EDA and Visualization

In [5]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [14]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [15]:
#melt_data(nv_df)

,value
time,
1996-04-01,118299.123063
1996-05-01,118419.044139
1996-06-01,118537.423268
1996-07-01,118653.069278
1996-08-01,118780.254312
...,...
2017-12-01,281095.320247
2018-01-01,282657.060382
2018-02-01,284368.688447


# Step 5: ARIMA Modeling

# Step 6: Interpreting Results